In [ ]:
!pip install --upgrade pip
!pip install konlpy  

!pip install nltk

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords  

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import json
import os
from tqdm import tqdm, tqdm_pandas

from konlpy.tag import Okt
from konlpy.tag import Mecab

import sklearn
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import log_loss, accuracy_score,f1_score
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from collections import Counter

from google.colab import drive

drive.mount('/content/drive')

In [ ]:
drive.mount('/content/drive')

In [89]:
file_path = '/content/drive/MyDrive/dataset/dacon/technic_classifier/'

train=pd.read_csv(file_path+'train.csv')
test=pd.read_csv(file_path+'test.csv')
sample_submission=pd.read_csv(file_path+'sample_submission.csv')

In [5]:
train.fillna('NoContent', inplace=True)
test.fillna('NoContent', inplace=True)

In [6]:
okt = Okt()
def trans_token(text):
  text = re.sub('가\.',' ',str(text))
  text = re.sub('나\.',' ',text)
  # text = re.sub('-\d+','num', text)
  text = re.sub('\<br\>',' ',text)
  text = re.sub('[\*\n]',' ',text)
  text = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z\- ]"," ", text)
  text = text.lower()
  if len(text) == 0:
    return ['NoContent']
  return okt.pos(text)

In [7]:
# 키워드는 대부분 품사가 명사임
tokens = okt.pos(train['과제명'].loc[0])
only_noun = [ x[0] for x in tokens if x[1] == 'Noun']
print(train['과제명'].loc[0])
print(only_noun)

유전정보를 활용한 새로운 해충 분류군 동정기술 개발
['유전', '정보', '활용', '해충', '분류군', '동정', '기술', '개발']


In [8]:
# 띄어쓰기와 전문용어가 많은 데이터 특성상 

tokens = okt.pos(train['과제명'].loc[21])
only_noun = [ x[0] for x in tokens if x[1] in ['Noun', 'Alpha']]
print(train['과제명'].loc[21])
print(only_noun)
print(okt.pos(train['과제명'].loc[21]))
# 띄어쓰기 문제로 한 단어가 Modifier(수식언,관형사)로 잘못 분리, 구조물 -> 구 + 조물

수지함침공정을 이용한 항공기 날개구조물 고굴곡 스파 개발
['함침', '공정', '이용', '항공기', '날개', '조물', '굴곡', '스파', '개발']
[('수', 'Modifier'), ('지', 'Modifier'), ('함침', 'Noun'), ('공정', 'Noun'), ('을', 'Josa'), ('이용', 'Noun'), ('한', 'Josa'), ('항공기', 'Noun'), ('날개', 'Noun'), ('구', 'Modifier'), ('조물', 'Noun'), ('고', 'Modifier'), ('굴곡', 'Noun'), ('스파', 'Noun'), ('개발', 'Noun')]


In [9]:
tokens = okt.pos(train['과제명'].loc[1])
only_noun = [ x[0] for x in tokens if x[1] in ['Noun', 'Alpha']]
print(train['과제명'].loc[1])
print(only_noun)
print(okt.pos(train['과제명'].loc[1]))
# 잘못된 Determiner(관형사) 분리로 내성이라는 고유의 의미를 갖는 명사가 내, 성으로 쪼개짐 

대장암의 TRAIL 내성 표적 인자 발굴 및 TRAIL 반응 예측 유전자 지도 구축에 관한 연구
['대장암', 'TRAIL', '성', '표적', '인자', '발굴', '및', 'TRAIL', '반응', '예측', '유전자', '지도', '구축', '관', '연구']
[('대장암', 'Noun'), ('의', 'Josa'), ('TRAIL', 'Alpha'), ('내', 'Determiner'), ('성', 'Noun'), ('표적', 'Noun'), ('인자', 'Noun'), ('발굴', 'Noun'), ('및', 'Noun'), ('TRAIL', 'Alpha'), ('반응', 'Noun'), ('예측', 'Noun'), ('유전자', 'Noun'), ('지도', 'Noun'), ('구축', 'Noun'), ('에', 'Josa'), ('관', 'Noun'), ('한', 'Josa'), ('연구', 'Noun')]


In [10]:
tokens = okt.pos(train['요약문_연구목표'].loc[13])
only_noun = [ x[0] for x in tokens if x[1] in ['Noun', 'Alpha']]
print(train['요약문_연구목표'].loc[13][140:180])
print(only_noun[30:40])
print(okt.pos(train['요약문_연구목표'].loc[13][140:175]))
# Modifier(수식언,관형사), Suffix(접미사) 잘못 분리/ 전자구조-> 전 + 자구 + 조

 촉매반응 과정 중에 일어나는 표면 전자구조변위의 동역학연구를 진행하고자
['촉매', '반응', '과정', '중', '표면', '자구', '변위', '동역학', '연구', '진행']
[('촉매', 'Noun'), ('반응', 'Noun'), ('과정', 'Noun'), ('중', 'Noun'), ('에', 'Josa'), ('일어나는', 'Verb'), ('표면', 'Noun'), ('전', 'Modifier'), ('자구', 'Noun'), ('조', 'Suffix'), ('변위', 'Noun'), ('의', 'Josa'), ('동역학', 'Noun'), ('연구', 'Noun'), ('를', 'Josa')]


In [43]:
kor_stop_words = ['후', '재', '통해', '통한','적','트','급','함','그','대한','은','상','인','무','피',
                  '용','활용','급','을','를','것','것임','임','기반','통한','별','때','외','못','기','년',
                  '차','간','등', '이','내','또','중','다른','역','일','제','통','시','총','상의','법','로',
                  '이들','물','개','년','건','주','사','점','계','각','더','뿐','안','두','유','과','와','덜',
                  '회','전','약','바','즉','신','데','좀','더','리','몇','최','번','움','줌','음','샤','셔',
                  '그것','형',
                  ]
custom_stop_words = ['주요','목표','개선','향상','과제','개발','기술','연구','선행','시스템','다음','매우','이용','로서','기존',
                     '극복','장점','단점','문제','문제점','각각','관련','부분','간의','이상','달성','주된내용','내용','국내',
                     '해외','최종','사용','효과','기관','제공','이하','목적','년차','필요','때문','로부터','최종적','만개','크게',
                     '모두','의미','여러','하나로','중심','한번','창','실','베','헤','핑','메','위해','차년','년도']

noun_char_to_front = ['러닝','품','체','층','능','브','스','물','팅','이드','버','리움','링','터','레이','베이','막','단','산',
                      '신','량','사체','액','망','켓','션','텍','틴','셋','론']

kor_stop_words = kor_stop_words + custom_stop_words
eng_stop_words = list(set(stopwords.words('english') + ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']))
stop_words = kor_stop_words + eng_stop_words

Modifier_to_front = ['칠','귀','속','별','너','두','흔','울','댓','성']
Modifier_no_change = ['동','총','순','양대', '일대','전전','다른','닷']
Modifier_to_Noun = ['암','즉석','기만','수조','온간','수삼']


def fixed_okt_tokenizer(text):

  tuple_tokens = text
  list_tokens = []
  for i in tuple_tokens:
    list_tokens.append(list(i))
  for i in range(len(list_tokens)):

    #보편적으로 작업해줘야하는 것
    if i < len(list_tokens) - 1:

      if (list_tokens[i+1][1] == 'Modifier') and (list_tokens[i+1][0] in Modifier_to_front):
        list_tokens[i][0] = list_tokens[i][0] + list_tokens[i+1][0]
        list_tokens[i][1] = 'Noun'
        list_tokens[i+1][1] = '제외'
        continue

      elif (list_tokens[i][1] == 'Modifier') and (list_tokens[i][0] not in (Modifier_to_front + Modifier_no_change + Modifier_to_Noun)):
        if (list_tokens[i][0] == '양') and (list_tokens[i-1][0] == '요' and list_tokens[i-1][1] == 'Josa'):
          list_tokens[i-1][0] = list_tokens[i-1][0] + list_tokens[i][0]
          list_tokens[i-1][1] = 'Noun'
          continue

        list_tokens[i+1][0] = list_tokens[i][0] + list_tokens[i+1][0]
        list_tokens[i+1][1] = 'Noun'
  
        continue

      elif (list_tokens[i][1] == 'Suffix') and (list_tokens[i][0] == '분') and list_tokens[i+1][0][0] in ['리','류','석']:
        list_tokens[i+1][0] = list_tokens[i][0] + list_tokens[i+1][0]
        list_tokens[i+1][1] = 'Noun'
        continue

      elif (list_tokens[i][1] == 'Determiner') and (list_tokens[i+1][1] == 'Noun'):
        list_tokens[i+1][0] = list_tokens[i][0] + list_tokens[i+1][0]
        list_tokens[i][1] = '제외'
        continue

      elif (list_tokens[i][0] in ['위','관'] and (list_tokens[i][1] == 'Noun')) and ((list_tokens[i+1][1] == 'Verb') or (list_tokens[i+1][1] == 'Josa')): # 위하여, 위한, 위해서는, 관하여, 관한, 관해서는 등 제거
        list_tokens[i][1] = '제외'
        continue

      elif ((list_tokens[i][0] == '및' or (list_tokens[i][0] == '의')) and (list_tokens[i][1] == 'Noun')): # 및 제거
        list_tokens[i][1] = '제외'
        continue

      elif (list_tokens[i][0] == ('초') and (list_tokens[i][1] == 'Noun')) and (list_tokens[i+1][1] == 'Noun'):
        list_tokens[i+1][0] = list_tokens[i][0] + list_tokens[i+1][0]
        list_tokens[i][1] = '제외'
        continue

      elif (list_tokens[i][0] in ['퇴','초','비','산'] and (list_tokens[i][1] == 'Noun')):
        if (list_tokens[i][0] == '산') and (list_tokens[i+1][0] != '업체'):
          continue 
        list_tokens[i][1] = '제외'
        list_tokens[i+1][0] = list_tokens[i][0] + list_tokens[i+1][0]
        # list_tokens[i+1][1] = 'Noun'
        continue

      elif (list_tokens[i][0] == '검' and (list_tokens[i][1] == 'Noun')):
        list_tokens[i][1] = '제외'
        list_tokens[i+1][1] = 'Noun'
        if list_tokens[i+1][0][0] == '출':
          list_tokens[i+1][0] = '검출'
          continue
        list_tokens[i+1][0] = list_tokens[i][0] + list_tokens[i+1][0]
        continue
      
      elif (list_tokens[i][0] == '변' and list_tokens[i][1] == 'Noun') and (list_tokens[i+1][0][0] == '이'):
        list_tokens[i][1] = '제외'
        list_tokens[i+1][0] = '변이'
        list_tokens[i+1][1] = 'Noun'
        continue

      elif (list_tokens[i][0] == '실' and list_tokens[i][1] == 'Noun') and (list_tokens[i+1][0][0] == '내'):
        list_tokens[i][1] = '제외'
        list_tokens[i+1][0] = list_tokens[i][0] + list_tokens[i+1][0]
        continue

      elif (list_tokens[i][0] == '할' and (list_tokens[i][1] == 'Verb')) and (list_tokens[i+1][0] == '수' and (list_tokens[i+1][1] == 'Noun')):
        list_tokens[i+1][1] = '제외'



    if (list_tokens[i][1] == 'Modifier') and (list_tokens[i][0] in Modifier_no_change):
      continue

    elif (list_tokens[i][1] == 'Modifier') and (list_tokens[i][0] in Modifier_to_Noun):
      list_tokens[i][1] = 'Noun'

    elif (list_tokens[i][1] == 'Suffix') or ((list_tokens[i][0] == '학') or (list_tokens[i][0] == '율') or (list_tokens[i][0] == '률')):
      if (list_tokens[i][0] in ['들']):
        continue
      list_tokens[i][0] = list_tokens[i-1][0] + list_tokens[i][0]
      list_tokens[i][1] = 'Noun'
      list_tokens[i-1][1] = 'Josa'
      
      
    #뒷 글자를 앞에 붙일 때 
    elif (list_tokens[i][0] == ('기후학') and (list_tokens[i][1] == 'Noun')) and (list_tokens[i-1][0] == ('법') and (list_tokens[i-1][1] == 'Noun')):
      list_tokens[i-1][0] = list_tokens[i-1][0] + list_tokens[i][0]
      list_tokens[i][1] = 'JOSA'

    elif (list_tokens[i][0] in noun_char_to_front) and (list_tokens[i][1] == 'Noun'):
      if (list_tokens[i][0] in ['물']) and (list_tokens[i-1][1] != 'Noun'):
        continue
      elif (list_tokens[i][0] == '버') and (list_tokens[i-1][0][-1] not in ['이','터']):
        continue
      elif (list_tokens[i][0] == '스') and (list_tokens[i-1][0][-1] not in ['레']):
        continue
      elif (list_tokens[i][0] == '신') and (list_tokens[i-1][0][-1] not in ['수']):
        continue
      elif (list_tokens[i][0] == '산') and (list_tokens[i-1][0] not in ['국내']):
        continue
      elif (list_tokens[i][0] == '사체') and (list_tokens[i-1][0] not in ['피전','피']):
        continue
      elif (list_tokens[i][0] == '러닝') and (list_tokens[i-1][0] not in ['딥']):
        continue

      elif (list_tokens[i][0] in ['레이','베이']) and (i != 0):
        list_tokens[i][0] = list_tokens[i-1][0] + list_tokens[i][0]
        list_tokens[i-1][1] = '제외'
        continue


      list_tokens[i][1] = '제외'
      list_tokens[i-1][0] = list_tokens[i-1][0] + list_tokens[i][0]
      if list_tokens[i][0] in ['이드','리움','링']:
        list_tokens[i-1][1] = 'Noun'


    elif (list_tokens[i][0] == ('열') and (list_tokens[i][1] == 'Noun')) and (list_tokens[i-1][1] == 'Noun'):
      list_tokens[i-1][0] = list_tokens[i-1][0] + list_tokens[i][0]
      list_tokens[i][1] = '제외'
    elif list_tokens[i][0][:2] == ('백신'):
      list_tokens[i][0] = '백신'
      list_tokens[i][1] = 'Noun' 
    elif (i != 0) and (i != len(list_tokens) - 1) and (list_tokens[i][0] == '-'):
      if (list_tokens[i-1][1] == 'Alpha') and (list_tokens[i+1][1] in ['Alpha','Number']):
        list_tokens[i-1][1] = '제외'
        list_tokens[i][1] = '제외'
        list_tokens[i+1][1] = 'Noun'
        list_tokens[i+1][0] = list_tokens[i-1][0] + list_tokens[i][0] + list_tokens[i+1][0]
    elif ((list_tokens[i][0] == '형') and (list_tokens[i][1] == 'Noun')) and (okt.pos(list_tokens[i-1][0])[-1][1] in ['Number','Alpha']):
      list_tokens[i][1] = '제외'
      list_tokens[i-1][0] = list_tokens[i-1][0] + list_tokens[i][0]
        

  tokens = []
  for word, tag in list_tokens : 
      if (tag in ['Noun', 'Alpha']) and (word not in stop_words):  #명사, 알파뱃만 사용
          tokens.append(word)
  return tokens

In [26]:
index = 21
tokens = okt.pos(train['과제명'].loc[index])
only_noun = [ x[0] for x in tokens if x[1] in ['Noun', 'Alpha']]
print('    원문 :',train['과제명'].loc[index])
print('okt 기본 :',okt.morphs(train['과제명'].loc[index]))
print('okt 명사 :',only_noun)
print('okt 튜닝 :',new_okt_tokenizer(trans_token(train['과제명'].loc[index])))

    원문 : 수지함침공정을 이용한 항공기 날개구조물 고굴곡 스파 개발
okt 기본 : ['수', '지', '함침', '공정', '을', '이용', '한', '항공기', '날개', '구', '조물', '고', '굴곡', '스파', '개발']
okt 명사 : ['함침', '공정', '이용', '항공기', '날개', '조물', '굴곡', '스파', '개발']
okt 튜닝 : ['수지', '함침', '공정', '항공기', '날개', '구조물', '고굴곡', '스파']


In [23]:
index = 1
tokens = okt.pos(train['과제명'].loc[index])
only_noun = [ x[0] for x in tokens if x[1] in ['Noun', 'Alpha']]
print('    원문 :',train['과제명'].loc[index])
print('okt 기본 :',okt.morphs(train['과제명'].loc[index]))
print('okt 명사 :',only_noun)
print('okt 튜닝 :',new_okt_tokenizer(trans_token(train['과제명'].loc[index])))

    원문 : 대장암의 TRAIL 내성 표적 인자 발굴 및 TRAIL 반응 예측 유전자 지도 구축에 관한 연구
okt 기본 : ['대장암', '의', 'TRAIL', '내', '성', '표적', '인자', '발굴', '및', 'TRAIL', '반응', '예측', '유전자', '지도', '구축', '에', '관', '한', '연구']
okt 명사 : ['대장암', 'TRAIL', '성', '표적', '인자', '발굴', '및', 'TRAIL', '반응', '예측', '유전자', '지도', '구축', '관', '연구']
okt 튜닝 : ['대장암', 'trail', '내성', '표적', '인자', '발굴', 'trail', '반응', '예측', '유전자', '지도', '구축']


In [24]:
index = 13
tokens = okt.pos(train['요약문_연구목표'].loc[index])
only_noun = [ x[0] for x in tokens if x[1] in ['Noun', 'Alpha']]
print('    원문 :',train['요약문_연구목표'].loc[index][141:200])
print('okt 기본 :',okt.morphs(train['요약문_연구목표'].loc[index])[58:76])
print('okt 명사 :',only_noun[30:40])
print('okt 튜닝 :',new_okt_tokenizer(trans_token(train['요약문_연구목표'].loc[index]))[25:40])

    원문 : 촉매반응 과정 중에 일어나는 표면 전자구조변위의 동역학연구를 진행하고자 한다.
okt 기본 : ['촉매', '반응', '과정', '중', '에', '일어나는', '표면', '전', '자구', '조', '변위', '의', '동역학', '연구', '를', '진행', '하고자', '한다']
okt 명사 : ['촉매', '반응', '과정', '중', '표면', '자구', '변위', '동역학', '연구', '진행']
okt 튜닝 : ['반응', '과정', '표면', '전자구조', '변위', '동역학', '진행']


In [18]:
num = 1706
tokens = okt.pos(train['요약문_연구목표'].loc[num])
only_noun = [ x[0] for x in tokens if x[1] in ['Noun', 'Alpha']]
print('    원문 :',train['요약문_연구목표'].loc[num][30:106])
print('okt 기본 :',okt.morphs(train['요약문_연구목표'].loc[num])[13:43])
print('okt 명사 :',only_noun[11:29])
print('okt 튜닝 :',new_okt_tokenizer(trans_token(train['요약문_연구목표'].loc[num]))[8:19])

    원문 :  드라이버 공항 배송 온디맨드(On-Demand) 서비스와 그것을 운영, 지휘하는 관제시스템인 오퍼레이팅 시스템을 개발 하고 보급하는 것
okt 기본 : ['드라이버', '공항', '배송', '온디맨드', '(', 'On', '-', 'Demand', ')', '서비스', '와', '그것', '을', '운영', ',', '지휘', '하는', '관제', '시스템', '인', '오퍼', '레이', '팅', '시스템', '을', '개발', '하고', '보급', '하는', '것']
okt 명사 : ['드라이버', '공항', '배송', '온디맨드', 'On', 'Demand', '서비스', '그것', '운영', '지휘', '관제', '시스템', '레이', '팅', '시스템', '개발', '보급', '것']
okt 튜닝 : ['드라이버', '공항', '배송', '온디맨드', 'on-demand', '서비스', '운영', '지휘', '관제', '오퍼레이팅', '보급']


In [32]:
index = 6

tokens = okt.pos(train['과제명'].loc[index])
only_noun = [ x[0] for x in tokens if x[1] in ['Noun', 'Alpha']]
print('    원문 :',train['과제명'].loc[index])
print('okt 기본 :',okt.morphs(train['과제명'].loc[index]))
print('okt 명사 :',only_noun)
print('okt 튜닝 :',new_okt_tokenizer(trans_token(train['과제명'].loc[index])))

# 앞, 뒤 어느쪽에 붙여야할지 애매한 단어, 따로 처리를 해준 단어만 적용이 되기 때문에 적용이 균일하지는 않음
# 시스를 앞에 붙이는 처리를 할 수도 있지만, 뒤에 붙이는 경우가 나을 수도 있고, 이런식으로 데이터 확인과 처리작업을 해줘야하는 케이스가 매우 많음 -> 토큰화 성능에 편차 발생 
# '필기 도구 외 기타' 같은 의존명사 외 제거하기 위해 외를 stop word에 추가하여 토큰 삭제가 발생

    원문 : 마이크로시스를 적용한 옥내·외 겸용 광케이블 개발
okt 기본 : ['마이크로', '시스', '를', '적용', '한', '옥내', '·', '외', '겸용', '광케이블', '개발']
okt 명사 : ['마이크로', '시스', '적용', '옥내', '외', '겸용', '광케이블', '개발']
okt 튜닝 : ['마이크로', '시스', '적용', '옥내', '겸용', '광케이블']


In [34]:
def united_contents(text):
  text = re.sub('\(R&D\)','',text)
  text = re.sub('\(\w+\)','',text)
  text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z ]','',text)
  return [text]

train['united_내역사업명'] = train['내역사업명'].apply(united_contents)
test['united_내역사업명'] = test['내역사업명'].apply(united_contents)

In [ ]:
tqdm_pandas(tqdm())

train['token_내역사업명'] = train['united_내역사업명'].progress_apply(trans_token)
train['token_과제명'] = train['과제명'].progress_apply(trans_token)
train['token_요약문_한글키워드'] = train['요약문_한글키워드'].progress_apply(trans_token)
train['token_요약문_연구목표'] = train['요약문_연구목표'].progress_apply(trans_token)

test['token_내역사업명'] = test['united_내역사업명'].progress_apply(trans_token)
test['token_과제명'] = test['과제명'].progress_apply(trans_token)
test['token_요약문_한글키워드'] = test['요약문_한글키워드'].progress_apply(trans_token)
test['token_요약문_연구목표'] = test['요약문_연구목표'].progress_apply(trans_token)

In [ ]:
tqdm_pandas(tqdm())

train['fixed_okt_내역사업명'] = train['token_내역사업명'].progress_apply(fixed_okt_tokenizer)
train['fixed_okt_과제명'] = train['token_과제명'].progress_apply(fixed_okt_tokenizer)
train['fixed_okt_요약문_한글키워드'] = train['token_요약문_한글키워드'].progress_apply(fixed_okt_tokenizer)
train['fixed_okt_요약문_연구목표'] = train['token_요약문_연구목표'].progress_apply(fixed_okt_tokenizer)

test['fixed_okt_내역사업명'] = test['token_내역사업명'].progress_apply(fixed_okt_tokenizer)
test['fixed_okt_과제명'] = test['token_과제명'].progress_apply(fixed_okt_tokenizer)
test['fixed_okt_요약문_한글키워드'] = test['token_요약문_한글키워드'].progress_apply(fixed_okt_tokenizer)
test['fixed_okt_요약문_연구목표'] = test['token_요약문_연구목표'].progress_apply(fixed_okt_tokenizer)

In [45]:
train1 = train[['index','fixed_okt_내역사업명','fixed_okt_과제명','fixed_okt_요약문_한글키워드','fixed_okt_요약문_연구목표','label','token_요약문_연구목표']]
test1 = test[['index','fixed_okt_내역사업명','fixed_okt_과제명','fixed_okt_요약문_한글키워드','fixed_okt_요약문_연구목표','token_요약문_연구목표']]

train1.to_csv('fixed_okt_train.csv',index=False)
test1.to_csv('fixed_test.csv',index=False)

In [101]:
train=pd.read_csv(file_path+'fixed_okt_train.csv')
test=pd.read_csv(file_path+'fixed_okt_test.csv')

In [104]:
# csv에는 리스트로 저장했던 것들이 문자열로 바뀌어있었으므로 문자열을 다시 리스트 형태로 변환
def trans_to_list(text):
  text = re.sub("[\[\]\' ]", "", str(text))
  text =  text.split(',')
  if text == ['']:
    text = ['']
  return text

train['fixed_okt_과제명'] = train['fixed_okt_과제명'].apply(trans_to_list)
train['fixed_okt_내역사업명'] = train['fixed_okt_내역사업명'].apply(trans_to_list)
train['fixed_okt_요약문_한글키워드'] = train['fixed_okt_요약문_한글키워드'].apply(trans_to_list)
train['fixed_okt_요약문_연구목표'] = train['fixed_okt_요약문_연구목표'].apply(trans_to_list)

test['fixed_okt_과제명'] = test['fixed_okt_과제명'].apply(trans_to_list)
test['fixed_okt_내역사업명'] = test['fixed_okt_내역사업명'].apply(trans_to_list)
test['fixed_okt_요약문_한글키워드'] = test['fixed_okt_요약문_한글키워드'].apply(trans_to_list)
test['fixed_okt_요약문_연구목표'] = test['fixed_okt_요약문_연구목표'].apply(trans_to_list)

In [105]:
# 더 제외하고 싶은 단어 추가 

del_word = [
            '안팎','개념','역할','중심적','또한','특이적','개월','취급','변화',
            '고려','가량','일반적','지원','다기','관','종합적','도출','방안','검토',
            '산업','강화','기업','바탕','현장','조건','방법','성숙','해당과정','위해',
            '검토','확립','차년','진행','과정','구축','연구개발','결과물',
            '검증','방안','인재','분석','발굴','규명','적용','실제','거나','연구원',
            '운영','확보','대하','수','능력','평가','성과','발표','도록','마련','고',
            '따라서','대해','제시','통합적','종합적','말함','살피','최근','대두','일반',
            '발전','연구개발','경우','주어','기초','년간','담당','낼','대처','성의','핵심',
            '주목표','상위','하위','기대','시기','증대','증가','감소','고자','이바지',
            '이전','현재', '이번',
]

def add_filter(text):
  return [x for x in text if x not in del_word]

train['filter_okt_요약문_연구목표'] = train['fixed_okt_요약문_연구목표'].apply(add_filter)
test['filter_okt_요약문_연구목표'] = test['fixed_okt_요약문_연구목표'].apply(add_filter)

In [106]:
train['vocab'] = train['fixed_okt_내역사업명'] + train['fixed_okt_과제명']  + train['fixed_okt_요약문_한글키워드'] + train['filter_okt_요약문_연구목표']

In [107]:
tokenizer=Tokenizer(oov_token = 'OOV')

tokenizer.fit_on_texts(list(train['vocab']))
word_vocab = tokenizer.word_index

vocab_size = len(word_vocab)+2

In [112]:
threshold = 10
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

print('예상 단어집합 크기 :',total_cnt - rare_cnt)
words_num = total_cnt - rare_cnt

단어 집합(vocabulary)의 크기 : 165958
등장 빈도가 9번 이하인 희귀 단어의 수: 121699
단어 집합에서 희귀 단어의 비율: 73.33120428060111
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 2.6347518664728007
예상 단어집합 크기 : 44259


In [113]:
# tokenizer=Tokenizer()
tokenizer=Tokenizer(num_words=words_num,oov_token = 'OOV')

tokenizer.fit_on_texts(list(train['vocab']))
word_vocab = tokenizer.word_index

vocab_size = len(word_vocab)+2

In [115]:
train['input_0'] = train['fixed_okt_내역사업명'] + train['fixed_okt_과제명'] + train['fixed_okt_요약문_한글키워드']
train['input_1'] = train['filter_okt_요약문_연구목표']

In [116]:
test['input_0'] = test['fixed_okt_내역사업명'] + test['fixed_okt_과제명'] + test['fixed_okt_요약문_한글키워드']
test['input_1'] = test['filter_okt_요약문_연구목표']

In [117]:
train_length = train['input_0'].apply(len)
train_length.head()

# 사분위의 대한 경우는 0~100 스케일로 되어있음
print('인풋 길이 제 1 사분위: {}'.format(np.percentile(train_length, 25)))
print('인풋 길이 제 3 사분위: {}'.format(np.percentile(train_length, 75)))

number = 98
print('인풋 길이 {}%: {}'.format(number,np.percentile(train_length, number)))

인풋 길이 제 1 사분위: 17.0
인풋 길이 제 3 사분위: 25.0
인풋 길이 98%: 35.0


In [118]:
train_length = train['input_1'].apply(len)
train_length.head()

# 사분위의 대한 경우는 0~100 스케일로 되어있음
print('인풋 길이 제 1 사분위: {}'.format(np.percentile(train_length, 25)))
print('인풋 길이 제 3 사분위: {}'.format(np.percentile(train_length, 75)))

number = 98
print('인풋 길이 {}%: {}'.format(number,np.percentile(train_length, number)))

인풋 길이 제 1 사분위: 23.0
인풋 길이 제 3 사분위: 68.0
인풋 길이 98%: 170.0


In [119]:
# 0이 아닌 데이터 2번 복사하여 추가, 추가로 오답률이 높은 label 2번 복사하여 추가 
temp = train.copy()
temp1 = temp[temp['label'] != 0]
temp2 = temp[(temp['label'] == 11)|(temp['label'] == 12)|(temp['label'] == 13)|(temp['label'] == 17)|(temp['label'] == 20)|(temp['label'] == 23)|(temp['label'] == 25)|(temp['label'] == 26)|(temp['label'] == 37)|(temp['label'] == 38)]

temp = pd.concat([temp, temp1], axis = 0).reset_index(drop = True)
temp = pd.concat([temp, temp1], axis = 0).reset_index(drop = True)
temp = pd.concat([temp, temp2], axis = 0).reset_index(drop = True)
temp = pd.concat([temp, temp2], axis = 0).reset_index(drop = True)

In [120]:
max_len = [35, 170]


# train_encoded_0 = tokenizer.texts_to_sequences(list(train['input_0']))
train_encoded_0 = tokenizer.texts_to_sequences(list(temp['input_0']))
train_input_0 = pad_sequences(train_encoded_0, maxlen=max_len[0], padding='post')

# train_encoded_1 = tokenizer.texts_to_sequences(list(train['input_1']))
train_encoded_1 = tokenizer.texts_to_sequences(list(temp['input_1']))
train_input_1 = pad_sequences(train_encoded_1, maxlen=max_len[1], padding='post')

train_input_0 = tf.convert_to_tensor(train_input_0, dtype=tf.float32)
train_input_1 = tf.convert_to_tensor(train_input_1, dtype=tf.float32)

train_input = []
train_input.append(train_input_0)
train_input.append(train_input_1)

In [121]:
test_encoded_0 = tokenizer.texts_to_sequences(list(test['input_0']))
test_input_0 = pad_sequences(test_encoded_0, maxlen=max_len[0], padding='post')

test_encoded_1 = tokenizer.texts_to_sequences(list(test['input_1']))
test_input_1 = pad_sequences(test_encoded_1, maxlen=max_len[1], padding='post')

test_input_0 = tf.convert_to_tensor(test_input_0, dtype=tf.float32)
test_input_1 = tf.convert_to_tensor(test_input_1, dtype=tf.float32)

test_input = []
test_input.append(test_input_0)
test_input.append(test_input_1)

In [137]:
def trans_label(number):
    vector =  np.array([0] * 46)
    vector[number] = 1
    return vector

# train['label_vector'] = train['label'].apply(trans_label)
# labels = tf.convert_to_tensor(list(train['label_vector']), dtype=tf.float32)

temp['label_vector'] = temp['label'].apply(trans_label)
labels = tf.convert_to_tensor(list(temp['label_vector']), dtype=tf.float32)

In [134]:
embedding_dim = [512, 512]

def convolutional_model(embedding_dim = embedding_dim, max_len = max_len, num_labels = 46):
    
    train_input_0 = tf.keras.Input(shape=(max_len[0],))
    train_input_1 = tf.keras.Input(shape=(max_len[1],))


    x0 = tf.keras.layers.Embedding(vocab_size, embedding_dim[0], input_length=max_len[0], mask_zero=True)(train_input_0)
    x0 = tf.keras.layers.Dropout(0.2)(x0)

    x0 = tf.keras.layers.Conv1D(embedding_dim[0], kernel_size=3, padding='valid', activation=None)(x0)
    x0 = tf.keras.layers.BatchNormalization()(x0)
    x0 = tf.keras.layers.Activation('relu')(x0)
    x0 = tf.keras.layers.Dropout(0.2)(x0)
    x0 = tf.keras.layers.GlobalMaxPooling1D()(x0)

    x0 = tf.keras.layers.Dense(128, activation=None)(x0)
    x0 = tf.keras.layers.BatchNormalization()(x0)
    x0 = tf.keras.layers.Activation('relu')(x0)
    x0 = tf.keras.layers.Dropout(0.2)(x0)

    # x0 = Dense(num_labels, activation = 'softmax')(x0)


    x1 = tf.keras.layers.Embedding(vocab_size, embedding_dim[1], input_length=max_len[1], mask_zero=True)(train_input_1)
    x1 = tf.keras.layers.Dropout(0.2)(x1)
    
    x1 = tf.keras.layers.Conv1D(embedding_dim[1], kernel_size=3, padding='valid', activation='relu')(x1)
    x1 = tf.keras.layers.BatchNormalization()(x1)
    x1 = tf.keras.layers.Activation('relu')(x1)
    x1 = tf.keras.layers.Dropout(0.2)(x1)
    x1 = tf.keras.layers.GlobalMaxPooling1D()(x1)

    x1 = tf.keras.layers.Dense(128, activation=None)(x1)
    x1 = tf.keras.layers.BatchNormalization()(x1)
    x1 = tf.keras.layers.Activation('relu')(x1)
    x1 = tf.keras.layers.Dropout(0.2)(x1)

    # x1 = Dense(num_labels, activation = 'softmax')(x1)

    fc = tf.keras.layers.Concatenate()([x0,x1])

    fc = tf.keras.layers.Dense(128, activation=None)(fc)
    fc = tf.keras.layers.BatchNormalization()(fc)
    fc = tf.keras.layers.Activation('relu')(fc)
    fc = tf.keras.layers.Dropout(0.2)(fc)

    labels = tf.keras.layers.Dense(num_labels, activation = 'softmax')(fc)
    
    model = tf.keras.Model([train_input_0,train_input_1], labels)
    optimizer = tf.keras.optimizers.Adam()
    model.compile(optimizer = optimizer, loss = 'CategoricalCrossentropy', metrics = ['accuracy'])
    
    return model

In [135]:
from PIL import Image

model = convolutional_model()
tf.keras.utils.plot_model(model, dpi = 500, show_shapes=True)
img = Image.open('model.png')
img

In [138]:
es = EarlyStopping(monitor = 'val_accuracy', mode = 'max', verbose = 1, patience = 15)
mc = ModelCheckpoint('best_model.h5', monitor = 'val_accuracy', mode = 'max', verbose = 1, save_best_only = True)

model = convolutional_model()


num_epochs = 60

history = model.fit(train_input, labels, epochs=num_epochs, batch_size=4096, verbose=1, validation_split=0.2, callbacks=[es, mc])

Epoch 1/60
49/49 [==============================] - 132s 2s/step - loss: 1.8256 - accuracy: 0.6451 - val_loss: 4.2398 - val_accuracy: 0.0000e+00

Epoch 00001: val_accuracy improved from -inf to 0.00000, saving model to best_model.h5
Epoch 2/60
49/49 [==============================] - 105s 2s/step - loss: 0.4967 - accuracy: 0.8971 - val_loss: 5.7293 - val_accuracy: 0.0000e+00

Epoch 00002: val_accuracy did not improve from 0.00000
Epoch 3/60
49/49 [==============================] - 104s 2s/step - loss: 0.2376 - accuracy: 0.9464 - val_loss: 6.2668 - val_accuracy: 0.0000e+00

Epoch 00003: val_accuracy did not improve from 0.00000
Epoch 4/60
49/49 [==============================] - 104s 2s/step - loss: 0.1385 - accuracy: 0.9688 - val_loss: 7.0444 - val_accuracy: 0.0000e+00

Epoch 00004: val_accuracy did not improve from 0.00000
Epoch 5/60
49/49 [==============================] - 105s 2s/step - loss: 0.0951 - accuracy: 0.9789 - val_loss: 7.3162 - val_accuracy: 0.0000e+00

Epoch 00005: val_a

In [139]:
num_epochs = 5

history = model.fit(train_input, labels, epochs=num_epochs, batch_size=4096, verbose=1, validation_split=0.2, callbacks=[es, mc])

Epoch 1/5
49/49 [==============================] - 104s 2s/step - loss: 0.0135 - accuracy: 0.9951 - val_loss: 0.0239 - val_accuracy: 0.9899

Epoch 00001: val_accuracy did not improve from 0.98997
Epoch 2/5
49/49 [==============================] - 103s 2s/step - loss: 0.0138 - accuracy: 0.9953 - val_loss: 0.0308 - val_accuracy: 0.9868

Epoch 00002: val_accuracy did not improve from 0.98997
Epoch 3/5
49/49 [==============================] - 105s 2s/step - loss: 0.0134 - accuracy: 0.9952 - val_loss: 0.0252 - val_accuracy: 0.9885

Epoch 00003: val_accuracy did not improve from 0.98997
Epoch 4/5
49/49 [==============================] - 104s 2s/step - loss: 0.0134 - accuracy: 0.9952 - val_loss: 0.0245 - val_accuracy: 0.9903

Epoch 00004: val_accuracy improved from 0.98997 to 0.99033, saving model to best_model.h5
Epoch 5/5
49/49 [==============================] - 103s 2s/step - loss: 0.0136 - accuracy: 0.9953 - val_loss: 0.0283 - val_accuracy: 0.9895

Epoch 00005: val_accuracy did not improv

In [140]:
num_epochs = 5

history = model.fit(train_input, labels, epochs=num_epochs, batch_size=4096, verbose=1, validation_split=0.2, callbacks=[es, mc])

Epoch 1/5
49/49 [==============================] - 104s 2s/step - loss: 0.0131 - accuracy: 0.9955 - val_loss: 0.0369 - val_accuracy: 0.9846

Epoch 00001: val_accuracy did not improve from 0.99033
Epoch 2/5
49/49 [==============================] - 104s 2s/step - loss: 0.0127 - accuracy: 0.9952 - val_loss: 0.0256 - val_accuracy: 0.9900

Epoch 00002: val_accuracy did not improve from 0.99033
Epoch 3/5
49/49 [==============================] - 104s 2s/step - loss: 0.0133 - accuracy: 0.9952 - val_loss: 0.0247 - val_accuracy: 0.9895

Epoch 00003: val_accuracy did not improve from 0.99033
Epoch 4/5
49/49 [==============================] - 104s 2s/step - loss: 0.0136 - accuracy: 0.9950 - val_loss: 0.0221 - val_accuracy: 0.9906

Epoch 00004: val_accuracy improved from 0.99033 to 0.99057, saving model to best_model.h5
Epoch 5/5
49/49 [==============================] - 106s 2s/step - loss: 0.0128 - accuracy: 0.9952 - val_loss: 0.0220 - val_accuracy: 0.9915

Epoch 00005: val_accuracy improved from 

In [141]:
num_epochs = 5

history = model.fit(train_input, labels, epochs=num_epochs, batch_size=4096, verbose=1, validation_split=0.2, callbacks=[es, mc])

Epoch 1/5
49/49 [==============================] - 105s 2s/step - loss: 0.0128 - accuracy: 0.9952 - val_loss: 0.0286 - val_accuracy: 0.9888

Epoch 00001: val_accuracy did not improve from 0.99154
Epoch 2/5
49/49 [==============================] - 106s 2s/step - loss: 0.0134 - accuracy: 0.9951 - val_loss: 0.0252 - val_accuracy: 0.9891

Epoch 00002: val_accuracy did not improve from 0.99154
Epoch 3/5
49/49 [==============================] - 104s 2s/step - loss: 0.0130 - accuracy: 0.9953 - val_loss: 0.0279 - val_accuracy: 0.9893

Epoch 00003: val_accuracy did not improve from 0.99154
Epoch 4/5
49/49 [==============================] - 106s 2s/step - loss: 0.0125 - accuracy: 0.9953 - val_loss: 0.0194 - val_accuracy: 0.9925

Epoch 00004: val_accuracy improved from 0.99154 to 0.99252, saving model to best_model.h5
Epoch 5/5
49/49 [==============================] - 104s 2s/step - loss: 0.0123 - accuracy: 0.9954 - val_loss: 0.0244 - val_accuracy: 0.9908

Epoch 00005: val_accuracy did not improv

In [142]:
num_epochs = 5

history = model.fit(train_input, labels, epochs=num_epochs, batch_size=4096, verbose=1, validation_split=0.2, callbacks=[es, mc])

Epoch 1/5
49/49 [==============================] - 107s 2s/step - loss: 0.0124 - accuracy: 0.9954 - val_loss: 0.0212 - val_accuracy: 0.9919

Epoch 00001: val_accuracy did not improve from 0.99252
Epoch 2/5
49/49 [==============================] - 105s 2s/step - loss: 0.0122 - accuracy: 0.9953 - val_loss: 0.0224 - val_accuracy: 0.9916

Epoch 00002: val_accuracy did not improve from 0.99252
Epoch 3/5
49/49 [==============================] - 104s 2s/step - loss: 0.0122 - accuracy: 0.9954 - val_loss: 0.0225 - val_accuracy: 0.9903

Epoch 00003: val_accuracy did not improve from 0.99252
Epoch 4/5
49/49 [==============================] - 105s 2s/step - loss: 0.0124 - accuracy: 0.9952 - val_loss: 0.0251 - val_accuracy: 0.9903

Epoch 00004: val_accuracy did not improve from 0.99252
Epoch 5/5
49/49 [==============================] - 105s 2s/step - loss: 0.0124 - accuracy: 0.9955 - val_loss: 0.0251 - val_accuracy: 0.9901

Epoch 00005: val_accuracy did not improve from 0.99252


In [143]:
model.load_weights('best_model.h5')

In [144]:
# 제출 파일 작성
pred=model.predict(test_input)
pred=tf.argmax(pred, axis=1)
sample_submission['label']=pred
sample_submission.to_csv('convolutional_submission.csv', index=False)